In [41]:
# def imports():
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
# from category_encoders import OneHotEncoder, OrdinalEncoder
from xgboost import XGBRegressor
from shap import TreeExplainer, initjs, force_plot
from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
    
# imports()

In [42]:
train = pd.read_csv( 'train.csv' , index_col= 'id' )

test = pd.read_csv( 'test.csv' , index_col = 'id' )

In [43]:
train.rename( columns = { 'number_of_total_atoms' : 'tot_atom' , 
                          'percent_atom_al' : 'al_atom_p' ,
                          'percent_atom_ga' : 'ga_atom_p' , 
                          'percent_atom_in' : 'in_atom_p' , 
                          'lattice_vector_1_ang' : 'ang_1' ,
                          'lattice_vector_2_ang' : 'ang_2' , 
                          'lattice_vector_3_ang' : 'and_3' ,
                          'lattice_angle_alpha_degree' : 'angle_a' , 
                          'lattice_angle_beta_degree' : 'angle_b' ,
                          'lattice_angle_gamma_degree' : 'angle_g' ,
                          'formation_energy_ev_natom' : 'f_energy' ,
                          'bandgap_energy_ev' : 'bg_energy' } , inplace = True )

test.rename( columns = { 'number_of_total_atoms' : 'tot_atom' , 
                         'percent_atom_al' : 'al_atom_p' ,
                         'percent_atom_ga' : 'ga_atom_p' , 
                         'percent_atom_in' : 'in_atom_p' , 
                         'lattice_vector_1_ang' : 'ang_1' ,
                         'lattice_vector_2_ang' : 'ang_2' , 
                         'lattice_vector_3_ang' : 'and_3' ,
                         'lattice_angle_alpha_degree' : 'angle_a' , 
                         'lattice_angle_beta_degree' : 'angle_b' ,
                         'lattice_angle_gamma_degree' : 'angle_g' } , inplace = True )

def wrangle(df):

    atoms = [ 'al' , 'ga' , 'in' ]
    
    for elem in atoms:
        df[ f'{ elem }_atoms' ] = df.tot_atom * df[ f'{ elem }_atom_p' ]
    
    df[ 'al_ga_ratio' ] = df.al_atoms / df.ga_atoms
    df[ 'al_in_ratio' ] = df.al_atoms / df.in_atoms
    df[ 'in_ga_ratio' ] = df.in_atoms / df.ga_atoms

    ratios = [ 'al_ga_ratio' , 'al_in_ratio' , 'in_ga_ratio' ]
    nulls = [ np.nan , np.inf ]
    
    for col in ratios:
        df[ col ] = df[ col ].replace( np.inf , np.nan )
        df[ col ] = df[ col ].replace( np.nan , df[ col ].max() )
    
    return df

train = wrangle( train )

test = wrangle( test )

In [44]:
target1 = 'bg_energy'
target2 = 'f_energy'

X = train.drop( columns = [ target1 , target2 ] )
X_test = test

y1 = train[ target1 ]
y2 = train[ target2 ]

In [45]:
X1_train, X1_val, y1_train, y1_val = train_test_split( 
    X , y1 , train_size = 0.9 , random_state = 19 )

X2_train, X2_val, y2_train, y2_val = train_test_split( 
    X , y2 , train_size = 0.9 , random_state = 19 )

y1_pred = [y1_train.mean()] * len(y1_train)
y2_pred = [y2_train.mean()] * len(y2_train)

def baselines():
    print('Band Gap Energy')
    print('Mean (eV): ', y1_train.mean())
    print('Baseline MAE: ' , mean_absolute_error( y1_train , y1_pred ) )
    print()
    print('Formation Energy')
    print('Mean (eV): ', y2_train.mean())
    print('Baseline MAE: ' , mean_absolute_error( y2_train , y2_pred ) )

baselines()

Band Gap Energy
Mean (eV):  2.0804772685185187
Baseline MAE:  0.8267515736025377

Formation Energy
Mean (eV):  0.18696375
Baseline MAE:  0.08535369444444445


In [46]:
# Define models' and encoders' aliases    
ss = StandardScaler()

rscv =  RandomizedSearchCV
lr = LinearRegression()
r = Ridge( random_state = 19 )
rfr = RandomForestRegressor( random_state = 19 )
xgbr = XGBRegressor( random_state = 19 )
    
# Define models    
model_lr = make_pipeline( ss , lr )    
model_r = make_pipeline( r )
model_rfr = make_pipeline( rfr )
model_xgbr = make_pipeline( xgbr )

# Fit models to each target separately 
model_lr1 = model_lr.fit( X1_train , y1_train );    
model_r1 = model_r.fit( X1_train , y1_train );
model_rfr1 = model_rfr.fit( X1_train , y1_train );
model_xgbr1 = model_xgbr.fit( X1_train , y1_train );

model_lr2 = model_lr.fit( X2_train , y2_train );
model_r2 = model_r.fit( X2_train , y2_train );
model_rfr2 = model_rfr.fit( X2_train , y2_train );
model_xgbr2 = model_xgbr.fit( X2_train , y2_train );

In [47]:
def scores():
    print('Linear Regression')
    print('BandGap train MAE: ' , round( mean_absolute_error( 
                                     y1_train , model_lr1.predict( X1_train ) ) , 3 ) )
    print('BandGap val MAE: ' , round( mean_absolute_error( 
                                   y1_val , model_lr1.predict( X1_val ) ) , 3 ) )
    print()
    print('Formaion train MAE: ' , round( mean_absolute_error( 
                                      y2_train , model_lr2.predict( X2_train ) ) , 3 ) )
    print('Formation val MAE: ' , round( mean_absolute_error( 
                                     y2_val , model_lr2.predict( X2_val ) ) , 3 ) )
    print()

    print('Random Forest Regressor')
    print('BandGap train MAE: ' , round( mean_absolute_error( 
                                     y1_train , model_rfr1.predict( X1_train ) ) , 3 ) )
    print('BandGap val MAE: ' , round( mean_absolute_error( 
                                   y1_val , model_rfr1.predict( X1_val ) ) , 3 ) )
    print()
    print('Formaion train MAE: ' , round( mean_absolute_error( 
                                      y2_train , model_rfr2.predict( X2_train ) ) , 3 ) )
    print('Formation val MAE: ' , round( mean_absolute_error( 
                                     y2_val , model_rfr2.predict( X2_val ) ) , 3 ) )
    print()

    print('XGBRegressor')
    print('BandGap train MAE: ' , round( mean_absolute_error( 
                                     y1_train , model_xgbr1.predict( X1_train ) ) , 3 ) )
    print('BandGap val MAE: ' , round( mean_absolute_error( 
                                   y1_val , model_xgbr1.predict( X1_val ) ) , 3 ) )
    print()
    print('Formaion train MAE: ' , round( mean_absolute_error( 
                                      y2_train , model_xgbr2.predict( X2_train ) ) , 3 ) )
    print('Formation val MAE: ' , round( mean_absolute_error( 
                                     y2_val , model_xgbr2.predict( X2_val ) ) , 3 ) )
    
scores()

Linear Regression
BandGap train MAE:  1.895
BandGap val MAE:  1.859

Formaion train MAE:  0.059
Formation val MAE:  0.058

Random Forest Regressor
BandGap train MAE:  1.896
BandGap val MAE:  1.859

Formaion train MAE:  0.015
Formation val MAE:  0.027

XGBRegressor
BandGap train MAE:  1.896
BandGap val MAE:  1.859

Formaion train MAE:  0.013
Formation val MAE:  0.029


In [63]:
# Tuning models' parameters
# Parameter rages for linear regression...
fit_intercept = [ True ]
normalize = [ True ]
normalize_lr2 = [ True , False ]

# ...for ridge
alpha = np.arange( .25 , 1.75 , .05 )
max_iter = np.arange( 1640 , 1781 , 10 )
solver = [ 'sparse_cg' , 'saga' ]
normalize_r = [ False ]

# ...for random forest regressor
n_estimators = np.arange( 180 , 221 , 4 )
criterion = [ 'mse' , 'mae' ]
max_depth = np.arange( 9 , 13 , .5 )
max_depth_rfr2 = np.arange( 6 , 13 , 1 )
max_features = [ 'auto' , 'sqrt' , 'log2' ]
bootstrap = [ True ]
n_estimators_rfr2 = arange( 180 , 220 , 5)

# ... and for xgb regressor
objective = [ 'reg:gamma' , 
              'reg:squarederror' , 
              'reg:pseudohubererror']
eta = np.arange( .5 , .7 , .01 ) 
eta_xgbr2 = np.arange( .6 , .91 , .03 ) 
max_depth_x = np.arange( 1 , 3 , 1 ) 
subsample = np.arange( .8 , .91 , .0)
subsample_xgbr2 = np.arange( .78 , .88 , .01)
colsample_bytree = np.arange( .88 , .93 , .005)
colsample_bytree_xgbr2 = np.arange( .8 , .93 , .02)
tree_method = [ 'exact' , 'approx' ]
n_estimators_x = np.arange( 200 , 576 , 25 )
n_estimators_xgbr2 = np.arange( 120 , 321 , 20)
# Organize everything in new dictionaries
params_lr = { 'linearregression__fit_intercept' : fit_intercept ,
                  'linearregression__normalize' : normalize }
params_lr2 = { 'linearregression__fit_intercept': fit_intercept ,
                  'linearregression__normalize' : normalize_lr2 }
params_r = { 'ridge__alpha' : alpha ,
                 'ridge__fit_intercept' : fit_intercept ,
                 'ridge__normalize' : normalize_r ,
                 'ridge__max_iter' : max_iter ,
                 'ridge__solver' : solver }
params_rfr = { 'randomforestregressor__n_estimators' : n_estimators ,
                   'randomforestregressor__criterion' : criterion ,
                   'randomforestregressor__max_depth' : max_depth ,
                   'randomforestregressor__max_features' : max_features ,
                   'randomforestregressor__bootstrap' : bootstrap }
params_rfr2 = { 'randomforestregressor__n_estimators' : n_estimators_rfr2 ,
                   'randomforestregressor__criterion' : criterion ,
                   'randomforestregressor__max_depth' : max_depth_rfr2 ,
                   'randomforestregressor__max_features' : max_features ,
                   'randomforestregressor__bootstrap' : bootstrap }

params_xgbr = { 'xgbregressor__objective' : objective ,
                    'xgbregressor__eta' : eta ,
                    'xgbregressor__max_depth' : max_depth_x ,
                    'xgbregressor__subsample' : subsample ,
                    'xgbregressor__colsample_bytree' : colsample_bytree ,
                    'xgbregressor__n_estimators' : n_estimators_x ,
                    'xgbregressor__tree_method' : tree_method }

params_xgbr2 = { 'xgbregressor__objective' : objective ,
                    'xgbregressor__eta' : eta_xgbr2 ,
                    'xgbregressor__max_depth' : max_depth_x ,
                    'xgbregressor__subsample' : subsample_xgbr2 ,
                    'xgbregressor__colsample_bytree' : colsample_bytree_xgbr2 ,
                    'xgbregressor__n_estimators' : n_estimators_xgbr2 ,
                    'xgbregressor__tree_method' : tree_method }

# Define randomized search CV models
rscv_lr1 = RandomizedSearchCV( model_lr1 , 
                              params_lr ,
                              n_iter = 2 ,  
                              n_jobs = -1 , 
                              verbose = 2 ,
                              cv = 2 )
rscv_r1 = RandomizedSearchCV( model_r1 , 
                             params_r , 
                             n_iter = 50 ,  
                             n_jobs = -1 , 
                             verbose = 2 ,
                             random_state = 19 ,                            
                             cv = 3 )
rscv_rfr1 = RandomizedSearchCV( model_rfr1 , 
                               params_rfr , 
                               n_iter = 50 ,  
                               n_jobs = -1 , 
                               verbose = 2 ,
                               random_state = 19 ,                            
                               cv = 3 )
rscv_xgbr1 = RandomizedSearchCV( model_xgbr1 , 
                                params_xgbr , 
                                n_iter = 100 ,  
                                n_jobs = -1 , 
                                verbose = 2 ,
                                random_state = 19 ,                               
                                cv = 3 )

rscv_lr2 = RandomizedSearchCV( model_lr2 , 
                              params_lr2 ,
                              n_iter = 2 ,  
                              n_jobs = -1 , 
                              verbose = 2 ,
                              cv = 2 )
rscv_r2 = RandomizedSearchCV( model_r2 , 
                             params_r , 
                             n_iter = 20 ,  
                             n_jobs = -1 , 
                             verbose = 2 ,
                             random_state = 19 ,                            
                             cv = 3 )
rscv_rfr2 = RandomizedSearchCV( model_rfr2 , 
                               params_rfr2 , 
                               n_iter = 75 ,  
                               n_jobs = -1 , 
                               verbose = 2 ,
                               random_state = 19 ,                            
                               cv = 3 )
rscv_xgbr2 = RandomizedSearchCV( model_xgbr2 , 
                                params_xgbr2 , 
                                n_iter = 75 ,  
                                n_jobs = -1 , 
                                verbose = 2 ,
                                random_state = 19 ,                               
                                cv = 3 )

# Fit RCSV models with training data
rscv_lr1.fit( X1_train , y1_train );
rscv_r1.fit( X1_train , y1_train );
rscv_rfr1.fit( X1_train , y1_train );
rscv_xgbr1.fit( X1_train , y1_train );

rscv_lr2.fit( X2_train , y2_train );
rscv_r2.fit( X2_train , y2_train );
rscv_rfr2.fit( X2_train , y2_train );
rscv_xgbr2.fit( X2_train , y2_train );

Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
[19:43:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [67]:
# # Fine tune hyperparameters

# lr_models = [ rscv_lr1 , rscv_r1 , rscv_rfr1 , rscv_xgbr1 , 
#               rscv_lr2 , rscv_r2 , rscv_rfr2 , rscv_xgbr2 ]
# for mod in lr_models:
#     print( mod.best_params_ )
#     print()

227

In [ ]:
feature = 'al_atoms'
features = ['al_atoms', 'ga_atoms']

isolate1 = pdp_isolate( model = model_xgbr1 ,
                       dataset = X1_val ,
                       model_features = X1_val.columns ,
                       feature = feature )
pdp_plot( isolate1, feature_name = feature );

interact1 = pdp_interact( model = model_xgbr1 ,
                         dataset = X1_val ,
                         model_features = X1_val.columns ,
                         features = features )
pdp_interact_plot( interact1 , plot_type = 'grid' , feature_names = features );

In [ ]:
isolate2 = pdp_isolate( model = model_xgbr2 ,
                       dataset = X2_val ,
                       model_features = X2_val.columns ,
                       feature = feature )
pdp_plot( isolate2 , feature_name = feature );

interact2 = pdp_interact( model = model_xgbr2 ,
                         dataset = X2_val ,
                         model_features = X2_val.columns ,
                         features = features )
pdp_interact_plot( interact2 , plot_type = 'grid' , feature_names = features );